In [19]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import glob
import csv


In [66]:

def wind_rose_fig(frequencies_df, filename):
    fig = go.Figure()
    fig.add_trace(go.Barpolar(
        r=frequencies_df.loc[('0-2 m/s'), 'frequency'],
        name='1 m/s',
        marker_color='#bfe8ff'))

    fig.add_trace(go.Barpolar(
        r=frequencies_df.loc[('2-4 m/s'), 'frequency'],
        name='2 m/s',
        marker_color='#8eb9dd'))

    fig.add_trace(go.Barpolar(
        r=frequencies_df.loc[('4-6 m/s'), 'frequency'],
        name='3 m/s',
        marker_color='#77a2cd'))

    fig.add_trace(go.Barpolar(
        r=frequencies_df.loc[('6-8 m/s'), 'frequency'],
        name='4 m/s',
        marker_color='#5e89bc'))

    fig.add_trace(go.Barpolar(
        r=frequencies_df.loc[('8-10 m/s'), 'frequency'],
        name='5 m/s',
        marker_color='#4772ac'))

    fig.add_trace(go.Barpolar(
        r=frequencies_df.loc[('10-12 m/s'), 'frequency'],
        name='6 m/s',
        marker_color='#2e599b'))

    fig.add_trace(go.Barpolar(
        r=frequencies_df.loc[('12-14 m/s'), 'frequency'],
        name='7 m/s',
        marker_color='#17438b'))

    fig.add_trace(go.Barpolar(
        r=frequencies_df.loc[('14-16 m/s'), 'frequency'],
        name='8 m/s',
        marker_color='#002c7b'))

    fig.add_trace(go.Barpolar(
        r=frequencies_df.loc[('>16 m/s'), 'frequency'],
        name='>8 m/s',
        marker_color='#000000'))

    fig.update_traces(
        text=['North', 'NNE', 'NE', 'ENE', 'East', 'ESE', 'SE', 'SSE', 'South', 'SSW', 'SW', 'WSW', 'West', 'WNW', 'NW', 'NNW'])

    fig.update_layout(

        # title=title,
        # title_font_size=26,
        # title_x=0.463,
        # legend_font_size=18,
        showlegend=False,
        paper_bgcolor='rgba(0,0,0,0)',
        polar_bgcolor = "rgb(200,200,200)",
        polar_radialaxis_ticksuffix='%',
        polar_angularaxis_rotation=90,
        polar_angularaxis_direction='clockwise',
        polar_angularaxis_tickmode='array',
        polar_angularaxis_showticklabels=False,
        # polar_angularaxis_tickvals=[0, 22.5, 45, 72.5, 90, 112.5, 135, 157.5, 180, 202.5, 225, 247.5, 270, 292.5, 315, 337.5],
        # polar_angularaxis_ticktext=['<b>North</b>', 'NNE', '<b>NE</b>', 'ENE', '<b>East</b>', 'ESE', '<b>SE</b>', 'SSE', '<b>South</b>', 'SSW', '<b>SW</b>', 'WSW', '<b>West</b>', 'WNW', '<b>NW</b>', 'NNW'],
        # polar_angularaxis_tickfont_size=12,
        polar_radialaxis_tickfont_color= "rgb(20,20,20)",
        polar_radialaxis_tickmode='linear',
        polar_radialaxis_angle=-90,
        polar_radialaxis_tick0=10,
        polar_radialaxis_dtick=10,
        polar_radialaxis_tickangle=-90,
        polar_radialaxis_tickfont_size=14)

    fig.write_image('output/' + filename)
def wind_dir_speed_freq(boundary_lower_speed, boundary_higher_speed, boundary_lower_direction, boundary_higher_direction):
    # mask for wind speed column
    log_mask_speed = (wind_rose_data[:, 0] >= boundary_lower_speed) & (wind_rose_data[:, 0] < boundary_higher_speed)
    # mask for wind direction
    log_mask_direction = (wind_rose_data[:, 1] >= boundary_lower_direction) & (
                wind_rose_data[:, 1] < boundary_higher_direction)

    # application of the filter on the wind_rose_data array
    return wind_rose_data[log_mask_speed & log_mask_direction]    # application of the filter on the wind_rose_data array

def wind_rose(file):
    wind_rose_df = pd.DataFrame(np.zeros((16 * 9, 3)), index=None, columns=('direction', 'strength', 'frequency'))
    directions = ['N', 'NNE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE', 'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NNW']
    directions_deg = np.array([0, 22.5, 45, 72.5, 90, 112.5, 135, 157.5, 180, 202.5, 225, 247.5, 270, 292.5, 315, 337.5])
    speed_bins = ['0-2 m/s', '2-4 m/s', '4-6 m/s', '6-8 m/s', '8-10 m/s', '10-12 m/s', '12-14 m/s', '14-16 m/s', '>16 m/s']
    df = pd.read_csv(file, parse_dates=["DATE"],index_col=["DATE"], low_memory=False)
    df = df.drop(df.iloc[:,15:-1],axis = 1)
    filename = df['NAME'][2][:10] + '.png'
    lat = df['LATITUDE'][2]
    long = df['LONGITUDE'][2]
    df[['Dir','DirQ', 'Obs', 'Speed','SpeedQ']] = df['WND'].str.split(',', expand=True)
    df['Speed'] = df['Speed'].astype(int) * .1 #add decimal back in
    df['Speed'] = df['Speed'].astype(int)
    df['Dir'] = df['Dir'].astype(int)
    df = df.drop(df[(df.Speed == 9999)].index)
    df = df.drop(df[(df.Dir == 999)].index)
    wind_rose_data = df[['Speed', 'Dir']].to_numpy()
    step = 11.25
    for i in range(len(wind_rose_data)):
        if directions_deg[-1] + step <= wind_rose_data[i, 1] and wind_rose_data[i, 1] < 360:
            wind_rose_data[i, 1] = wind_rose_data[i, 1] - 360
    bin_edges_dir = directions_deg - step
    bin_edges_dir = np.append(bin_edges_dir, [directions_deg[-1] + step])
    threshold_outlier_rm = 9
    bin_edges_speed = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, threshold_outlier_rm])
    frequencies = np.array([])
    idx = pd.MultiIndex.from_product([speed_bins, directions_deg], names=['wind_speed_bins', 'wind_direction_bins'])
    col = ['frequency']
    frequencies_df = pd.DataFrame(0, idx, col)
    # loop selecting given bins and calculating frequencies
    for i in range(len(bin_edges_speed) - 1):
        for j in range(len(bin_edges_dir) - 1):
            bin_contents = wind_dir_speed_freq(bin_edges_speed[i], bin_edges_speed[i + 1], bin_edges_dir[j], bin_edges_dir[j + 1])

            # applying the filtering function for every bin and checking the number of measurements
            bin_size = len(bin_contents)
            frequency = bin_size / len(wind_rose_data)

            #obtaining the final frequencies of bin
            frequencies = np.append(frequencies, frequency)
    frequencies_df.frequency = frequencies * 100  # [%]
    wind_rose_df.frequency = frequencies * 100  # [%]
    wind_rose_fig(frequencies_df, filename)
    name_list = [lat, long, filename]
    return name_list




In [70]:

#csv_files = glob.glob('data/hawaii/*.csv')
year = "2022"
base_url = 'https://www.ncei.noaa.gov/data/global-hourly/access/'
location = '99819199999'
csv_file = base_url + year + '/' + location + '.csv'
name_list = []
for file in locations:
    name_list.append(wind_rose(file))


with open("output/file_list.csv", 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     for names in name_list:
         wr.writerow(names)